In [1]:
import os
try:
  import wget
except:
  !pip install wget
  import wget
import tarfile


out_dir = 'data/svhn'

train_32_32 = ('http://ufldl.stanford.edu/housenumbers/train_32x32.mat', 'train_32x32.mat')
test_32_32 = ('http://ufldl.stanford.edu/housenumbers/test_32x32.mat', 'test_32x32.mat')
extra_32_32 = ('http://ufldl.stanford.edu/housenumbers/extra_32x32.mat', 'extra_32x32.mat')

train_large = ('http://ufldl.stanford.edu/housenumbers/train.tar.gz', 'train.tar.gz')
test_large = ('http://ufldl.stanford.edu/housenumbers/test.tar.gz', 'test.tar.gz')
extra_large = ('http://ufldl.stanford.edu/housenumbers/extra.tar.gz', 'extra.tar.gz')

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=498001974df073ea383ccc2c479ed731cf5be245a51df7655736f766937a4ffe
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
def download_data(url, filename, out_dir=out_dir):
    filename = os.path.join(out_dir, filename)

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    if not os.path.exists(filename):
        print(f"Downloading {filename}.")
        wget.download(url, filename)
        print()
    else:
        print(f"Skipping {filename} download (already exists)")


def extract_data(filename, out_dir=out_dir):
    filename = os.path.join(out_dir, filename)

    print(f"Extracting {filename}")
    with tarfile.open(filename) as tar:
        tar.extractall(out_dir)

download_data(*train_32_32)
download_data(*test_32_32)
download_data(*extra_32_32)

download_data(*train_large)
download_data(*test_large)
# download_data(*extra_large)

extract_data(train_large[1])
extract_data(test_large[1])
# extract_data(extra_large[1])







Extracting data/svhn/train.tar.gz
Extracting data/svhn/test.tar.gz


In [3]:
from tensorflow import keras
import numpy as np
from PIL import Image
from pathlib import Path
from scipy import io

def to_one_hot(a, n):
    result = np.zeros(shape=(a.shape[0], n))
    result[np.arange(len(a)), a] = 1
    return result

def load_mnist():
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    def to_x(a):
        x = np.array([np.array(Image.fromarray(i).resize((32, 32))) for i in a])
        return x.reshape(x.shape + (1,))

    def to_y(a):
        return to_one_hot(a, 10)

    x_train, y_train = to_x(x_train), to_y(y_train)
    x_test, y_test = to_x(x_test), to_y(y_test)
    print('Loaded and processed mnist dataset')
    return x_train, y_train, x_test, y_test

def load_single_digit_data(dir='data/svhn', extra=False, greyscale=True):

    def to_x(a):
        a = np.array([a[:,:,:,i] for i in range(a.shape[3])])
        if greyscale:
            return np.mean(a, axis=-1, keepdims=True).astype(np.uint8)
        return a

    def to_y(a):
        y = np.copy(a)
        y = y.reshape(y.shape[0])
        y[y == 10] = 0
        return to_one_hot(y, 10)

    def load_file(file):
        cache_file = Path(dir) / f"{file}.cache.npz"
        if cache_file.exists():
            f = np.load(cache_file)
            print(f'Loaded cached arrays for {file}')
            return [v for k, v in f.items()]

        f = io.loadmat(Path(dir) / file)
        x, y = to_x(f['X']), to_y(f['y'])
        np.savez(Path(dir) / f"{file}.cache.npz", x, y)
        print(f'Loaded and processed {file}')
        return x, y
    x_train, y_train = load_file('train_32x32.mat')
    x_test, y_test = load_file('test_32x32.mat')

    x_extra, y_extra = None, None
    if extra:
        x_extra, y_extra = load_file('extra_32x32.mat')

    return (
        x_train, y_train,
        x_test, y_test,
        x_extra, y_extra
    )

x_train, y_train, x_test1, y_test1 = load_mnist()
_, _, x_test, y_test, _, _ = load_single_digit_data(extra=False)

11493376/11490434 [==============================] - 0s 0us/step
Loaded and processed mnist dataset
Loaded and processed train_32x32.mat
Loaded and processed test_32x32.mat


In [4]:
model = keras.Sequential([
            keras.layers.Conv2D(16, 5, activation='relu', input_shape=x_train.shape[1:], padding='same'),
            keras.layers.MaxPool2D(pool_size=(2, 2), padding = 'same'),
            keras.layers.Conv2D(32, 5, activation='relu', padding='same'),
            keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
            keras.layers.Conv2D(64, 5, activation='relu', padding='same'),
            keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
            keras.layers.Flatten(),
            keras.layers.Dropout(rate=0.1),
            keras.layers.Dense(100, activation='relu'),
            keras.layers.Dropout(rate=0.1),
            keras.layers.Dense(y_train.shape[1], activation='softmax')
        ])

model.compile(
            optimizer=keras.optimizers.Adam(0.001),
            loss='categorical_crossentropy',
            metrics=['categorical_accuracy']
        )
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = \
        train_test_split(x_train, y_train, test_size=0.1)

model.fit(
                x_train,
                y_train,
                epochs=100,
                verbose=2,
                batch_size=100,
                validation_data=(x_val, y_val),
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        patience=10,
                        restore_best_weights=True
                    )
                ]
            )

Epoch 1/100
540/540 - 2s - loss: 0.5141 - categorical_accuracy: 0.9156 - val_loss: 0.0654 - val_categorical_accuracy: 0.9793
Epoch 2/100
540/540 - 2s - loss: 0.0742 - categorical_accuracy: 0.9783 - val_loss: 0.0550 - val_categorical_accuracy: 0.9837
Epoch 3/100
540/540 - 2s - loss: 0.0555 - categorical_accuracy: 0.9829 - val_loss: 0.0388 - val_categorical_accuracy: 0.9863
Epoch 4/100
540/540 - 2s - loss: 0.0429 - categorical_accuracy: 0.9864 - val_loss: 0.0405 - val_categorical_accuracy: 0.9868
Epoch 5/100
540/540 - 2s - loss: 0.0414 - categorical_accuracy: 0.9874 - val_loss: 0.0576 - val_categorical_accuracy: 0.9847
Epoch 6/100
540/540 - 2s - loss: 0.0379 - categorical_accuracy: 0.9879 - val_loss: 0.0538 - val_categorical_accuracy: 0.9863
Epoch 7/100
540/540 - 2s - loss: 0.0317 - categorical_accuracy: 0.9901 - val_loss: 0.0407 - val_categorical_accuracy: 0.9875
Epoch 8/100
540/540 - 2s - loss: 0.0311 - categorical_accuracy: 0.9911 - val_loss: 0.0418 - val_categorical_accuracy: 0.9890


In [6]:
_, acc = model.evaluate(x_test, y_test)
print(f'Accuracy = {acc:.5f}')

814/814 [==============================] - 2s 2ms/step - loss: 2.2659 - categorical_accuracy: 0.1807
Accuracy = 0.18074


In [7]:
_, acc = model.evaluate(x_test1, y_test1)
print(f'Accuracy = {acc:.5f}')

313/313 [==============================] - 1s 2ms/step - loss: 0.0386 - categorical_accuracy: 0.9879
Accuracy = 0.98790


In [0]:
model.save_weights('models/svhn_mnist_conv_net_svhn/model')

In [9]:
!ls models/svhn_mnist_conv_net_svhn

checkpoint  model.data-00000-of-00002  model.data-00001-of-00002  model.index


In [10]:
x_train, y_train, x_test, y_test, _, _ = load_single_digit_data(extra=False)
model.fit(
                x_train,
                y_train,
                epochs=100,
                verbose=2,
                batch_size=100,
                validation_split=0.1,
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        patience=10,
                        restore_best_weights=True
                    )
                ]
            )

Loaded cached arrays for train_32x32.mat
Loaded cached arrays for test_32x32.mat
Epoch 1/100
660/660 - 3s - loss: 0.8766 - categorical_accuracy: 0.7268 - val_loss: 0.5117 - val_categorical_accuracy: 0.8438
Epoch 2/100
660/660 - 3s - loss: 0.4981 - categorical_accuracy: 0.8513 - val_loss: 0.4235 - val_categorical_accuracy: 0.8696
Epoch 3/100
660/660 - 3s - loss: 0.4174 - categorical_accuracy: 0.8761 - val_loss: 0.3886 - val_categorical_accuracy: 0.8842
Epoch 4/100
660/660 - 3s - loss: 0.3658 - categorical_accuracy: 0.8915 - val_loss: 0.3593 - val_categorical_accuracy: 0.8943
Epoch 5/100
660/660 - 3s - loss: 0.3342 - categorical_accuracy: 0.8995 - val_loss: 0.3515 - val_categorical_accuracy: 0.8972
Epoch 6/100
660/660 - 3s - loss: 0.3130 - categorical_accuracy: 0.9062 - val_loss: 0.3664 - val_categorical_accuracy: 0.8957
Epoch 7/100
660/660 - 3s - loss: 0.2899 - categorical_accuracy: 0.9120 - val_loss: 0.3680 - val_categorical_accuracy: 0.8975
Epoch 8/100
660/660 - 3s - loss: 0.2712 - ca

In [11]:
_, acc = model.evaluate(x_test1, y_test1)
print(f'Accuracy = {acc:.5f}')
_, acc = model.evaluate(x_test, y_test)
print(f'Accuracy = {acc:.5f}')

313/313 [==============================] - 1s 2ms/step - loss: 2.4196 - categorical_accuracy: 0.6049
Accuracy = 0.60490
814/814 [==============================] - 2s 2ms/step - loss: 0.3968 - categorical_accuracy: 0.8912
Accuracy = 0.89121


In [12]:
_, _, x_test, y_test, x_extra, y_extra = load_single_digit_data(extra=True)
model.fit(
                x_extra,
                y_extra,
                epochs=100,
                verbose=2,
                batch_size=100,
                validation_split=0.1,
                callbacks=[
                    keras.callbacks.EarlyStopping(
                        patience=10,
                        restore_best_weights=True
                    )
                ]
            )

Loaded cached arrays for train_32x32.mat
Loaded cached arrays for test_32x32.mat
Loaded and processed extra_32x32.mat
Epoch 1/100
4781/4781 - 19s - loss: 0.1914 - categorical_accuracy: 0.9470 - val_loss: 0.1345 - val_categorical_accuracy: 0.9621
Epoch 2/100
4781/4781 - 19s - loss: 0.1538 - categorical_accuracy: 0.9574 - val_loss: 0.1231 - val_categorical_accuracy: 0.9664
Epoch 3/100
4781/4781 - 19s - loss: 0.1389 - categorical_accuracy: 0.9615 - val_loss: 0.1174 - val_categorical_accuracy: 0.9680
Epoch 4/100
4781/4781 - 19s - loss: 0.1318 - categorical_accuracy: 0.9640 - val_loss: 0.1370 - val_categorical_accuracy: 0.9614
Epoch 5/100
4781/4781 - 19s - loss: 0.1273 - categorical_accuracy: 0.9648 - val_loss: 0.1126 - val_categorical_accuracy: 0.9694
Epoch 6/100
4781/4781 - 19s - loss: 0.1242 - categorical_accuracy: 0.9658 - val_loss: 0.1147 - val_categorical_accuracy: 0.9701
Epoch 7/100
4781/4781 - 19s - loss: 0.1211 - categorical_accuracy: 0.9669 - val_loss: 0.1090 - val_categorical_acc

In [13]:
_, acc = model.evaluate(x_test1, y_test1)
print(f'Accuracy = {acc:.5f}')
_, acc = model.evaluate(x_test, y_test)
print(f'Accuracy = {acc:.5f}')

313/313 [==============================] - 1s 2ms/step - loss: 4.1757 - categorical_accuracy: 0.4070
Accuracy = 0.40700
814/814 [==============================] - 2s 2ms/step - loss: 0.2789 - categorical_accuracy: 0.9296
Accuracy = 0.92959
